# Mark II

In [17]:
!pip install parmap

In [7]:
import pandas as pd
import numpy as np
import os 
from Bio import SeqIO
import csv
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

from multiprocessing import Pool

num_partitions = 18 #number of partitions to split dataframe
num_cores = 20 #number of cores on your machine


## Create Bad list

In [17]:
# construct badlist
amp_uniref_df = pd.read_csv('/mnt/vdb/thesis/uniref50/amp.forNONAMP.uniref50_2400.m8', sep='\t', header=0)
amp_uniref_df

,ID_Query,ID_Target,sequence_identity,alignment_length,number_of_mismatches,number_of_gap_openings,start-position_in_query,end-position_in_query,start-position_in_target,end-position_in_target,E-value,bit_score
0,EN92515250|C|B3FJD7|phage,UniRef50_B3FJD7,1.000,2337,0,0,1,2337,1,2337,0.000000e+00,4547
1,EN92515250|C|B3FJD7|phage,UniRef50_A0A1Y0SZ10,0.466,2337,1242,0,1,2337,1,2326,0.000000e+00,1858
2,EN92515250|C|B3FJD7|phage,UniRef50_F8SJ56,0.416,2336,1353,0,1,2336,1,2318,0.000000e+00,1628
3,EN92515250|C|B3FJD7|phage,UniRef50_A0A481W507,0.365,2135,1334,0,33,2134,22,2156,0.000000e+00,1171
4,EN92515250|C|B3FJD7|phage,UniRef50_Q8SCY1,0.349,2141,1346,0,1,2141,1,2069,0.000000e+00,1119
...,...,...,...,...,...,...,...,...,...,...,...,...
10191502,jgi|Copscl1|604239|estExt_Genemark1.C_650036,UniRef50_UPI000E6E4B9B,0.726,73,20,0,59,131,2,74,2.754000e-23,104
10191503,jgi|Copscl1|604239|estExt_Genemark1.C_650036,UniRef50_A0A3B5QMM9,0.647,85,29,0,24,107,17,101,9.724000e-23,102
10191504,jgi|Copscl1|604239|estExt_Genemark1.C_650036,UniRef50_B4IPI1,0.724,69,19,0,63,131,1,69,1.827000e-22,102
10191505,jgi|Copscl1|604239|estExt_Genemark1.C_650036,UniRef50_A8JIV7,0.536,97,44,0,35,130,13,109,1.827000e-22,102


In [18]:
bad_list = list(set(amp_uniref_df[amp_uniref_df['sequence_identity'] >=0.3]['ID_Target'].tolist())) 
len(bad_list)

304822

In [ ]:
bad_list

In [20]:
with open('/mnt/vdb/thesis/uniref50/nonAMP_Uniref_bad_list.txt', 'w') as f:
    for item in bad_list:
        f.write("%s\n" % item)

In [23]:
if "UniRef50_B4IPI1" in bad_list:
    print("True")

True


## Begin non-AMP Search

In [5]:
amp_fasta = '/mnt/vdb/thesis/amp.final.ready.fasta'
nonAMPs_csv = "/mnt/vdb/thesis/non_AMPs.3.csv"
uniref50 = "/mnt/vdb/thesis/uniref50/uniref50.excludedBadList.len2400.fasta"

In [37]:
with open(amp_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

AMPs_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
AMPs_df # 44346

,ID,Sequence,length
0,EN92515250|C|B3FJD7|phage,MAKKSVPLRKPAGSDGQGNIKVPGGPVVLDLGDFDDIFGPMESESP...,2337
1,EN54061055|C|F8SJ56|phage,MASKKTTLPKPKGINPQGSIVQLDLDDFDDLFDEDFGLPKKNSPYT...,2319
2,EN4815120|C|Q8SCY1|phage,MAKKVTLPKGQTGATGTTLGQAGNILDLSDVDDIFGDTPKAKKGSP...,2237
3,EN3016141|D|D2J8A7|bacteriocin,MAETIKGLRIDLSLKDMGVGRSITELKRSFRTLNSDLKVSSKNFEY...,1619
4,EN9175723|BD|Q93IM3|bacteriocin,MAKKKNTYKVPSIIALTLAGTALTTHHAQAADKTQDQSTNKNILND...,1564
...,...,...,...
44341,jgi|Zymar1|771619|Za111_scaffold_384_0026,MTKRTKKVGITGKYGTRYGASLRKQVKKIEISQHARYTCTFCGKVS...,92
44342,jgi|Zymbr1|3072|TI39_contig368g00016T0,MSIWLERFSAPSREQTPSPNPNRPYPASRRSAQLGPSTVPRRPGLQ...,769
44343,jgi|Zymps1|802458|Zp221_scaffold_490_00086,MAPKAQKTPTTAGKAPAGKAPAEKKEAGKKTAAATGDKKKRTKTRK...,138
44344,jgi|Zymps1|799992|Zp221_scaffold_243_00026,MQIFVKTLTGKTITLEVESSDTIDNVKSKIQDKEGIPPDQQRLIFA...,155


In [38]:
df_count=AMPs_df.groupby('length').length.count().reset_index(name='count').sort_values(['count'], ascending=False)
#df_count.to_csv('df_count.csv', index=False)
_tmp_df_count =df_count
_tmp_df_count

,length,count
9,20,1131
2,13,1119
1,12,912
7,18,873
4,15,855
...,...,...
728,836,1
729,839,1
730,840,1
731,842,1


In [39]:
_tmp_df_count = _tmp_df_count.sample(frac=1).reset_index(drop=True)

In [40]:
_tmp_df_count["count"].values

array([   1,   16,   16,    1,    1,  255,    5,    7,  201,    1,    1,
          6,    8,    8,    6,    7,   16,   22,    2,   15,   13,    8,
          8,    3,   16,    1,    1,    7,    3,   15,   22,    3,   15,
       1119,    2,   70,   33,   38,    1,   13,    4,    7,   53,    3,
        217,   12,  151,    1,    1,    1,    5,    9,   10,    6,   12,
          1,    3,  231,   14,    5,    3,   22,   36,    6,    1,   12,
          6,  353,   18,  217,    2,    7,   15,    4,    1,   19,    3,
          2,  310,   15,    1,    8,  912,    2,    8,    1,  359,    2,
          8,  125,   16,   26,    7,    1,   85,   31,    2,  405,    1,
          9,   17,   39,    1,    3,   14,   15,   12,   17,  285,    1,
         41,   20,   22,   10,   18,    5,    1,   13,  190,   23,    5,
        208,   19,   18,   26,   23,   28,   17,    1,   78,    1,   28,
          1,    2,    2,  559,    1,    1,    1,   54,   16,    9,    9,
          1,   25,   61,   13,    9,    1,    2,   

In [44]:

num_partitions = 23 #number of partitions to split dataframe 22
num_cores = 23 #number of cores on your machine 22

In [ ]:
"""
def parallelize_dataframe(df, func):
    _tmp_df_split = np.array_split(df, num_partitions)
    non_amp_data_df = pd.DataFrame(columns=['ID','Sequence','length']) 

    _list =[ [non_amp_data_df , _tmp_df_split] ]
    
    pool = Pool(num_cores)
    _list = pool.map(func,non_amp_data_df,_tmp_df_split)
    #non_amp_data_df ,_tmp_df_count = _list[0], _list[1]
    
   # non_amp_data_df = pd.concat(non_amp_data_df)
   # _tmp_df_count = pd.concat(_tmp_df_count)
    pool.close()
    pool.join()
    return _tmp_df_count, non_amp_data_df

def find_amp(_tmp_non_amp_data_df, _tmp_df_split):
    #_tmp_non_amp_data_df, _tmp_df_count = _list[0], _list[1]
    #_tmp_df_count = pd.concat(_tmp_df_count)
    #print(type(_tmp_non_amp_data_df))
    #print("-------------------")
    #print(_tmp_df_count)
    #NonAMPs_csv = open(nonAMPs_csv, 'w')
    #NonAMPs_csv.write('"ID","Sequence","length"'+ '\n')
    record = SeqIO.parse(uniref50, "fasta")
    for seq_record in record:

        seqLen = len(str(seq_record.seq))
        # update value
        _tmp_row = _tmp_df_count.loc[ _tmp_df_count['length'] == seqLen]
        if not _tmp_row.empty and  _tmp_row['count'].values > 0:
            print (str(_tmp_row['length']) +" "+ str(_tmp_row['count']))
          #  print('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"')
           # NonAMPs_csv.write('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"' + '\n')
            _tmp_non_amp_data_df.append({'ID': str(seq_record.id), 'Sequence': str(seq_record.seq)
                                     , 'length': str(seqLen)},ignore_index=True)
            _tmp_df_count.at[ _tmp_row.index,'count']= _tmp_row['count'] - 1
        #elif : # shulff seqeunce and subseq
        else:
            if _tmp_df_count['count'].values.sum() == 0:
                print("stop: found every range")
                break
    #NonAMPs_csv.close()
    _list =   [_tmp_non_amp_data_df, _tmp_df_count] 
    print(_list)
    return _list
"""

In [49]:
from os import getpid

def find_amp(_tmp_df_count):
    process_id =str(getpid())
    nonAMPs_csv = "/mnt/vdb/thesis/non_amp/non_amp."+process_id+".csv"
    NonAMPs_csv = open(nonAMPs_csv, 'w')
    NonAMPs_csv.write('"ID","Sequence","length"'+ '\n')
    print(process_id+" Start")
    record = SeqIO.parse(uniref50, "fasta")
    for seq_record in record:

        seqLen = len(str(seq_record.seq))
        # update value
        _tmp_row = _tmp_df_count.loc[ _tmp_df_count['length'] == seqLen]
        if not _tmp_row.empty and  _tmp_row['count'].values > 0:
           # print (str(_tmp_row['length']) +" "+ str(_tmp_row['count']))
           # print('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"')
            NonAMPs_csv.write('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"' + '\n')
            
            _tmp_df_count.at[ _tmp_row.index,'count']= _tmp_row['count'] - 1
        #elif : # shulff seqeunce and subseq
                    
        else:
            if _tmp_df_count['count'].values.sum() == 0:
                print(process_id+" stop: found every range")
                break
    NonAMPs_csv.close()


from functools import partial

def parallelize_dataframe(df, func):
    _tmp_df_split = np.array_split(df, num_partitions)
        
    pool = Pool(num_cores)
    pool.map(func, _tmp_df_split)

    pool.close()
    pool.join()


In [50]:
parallelize_dataframe(_tmp_df_count, find_amp)

5653 Start5649 Start5650 Start5651 Start5652 Start5657 Start5660 Start5656 Start

5655 Start5654 Start5662 Start5661 Start



5658 Start5659 Start


5648 Start5664 Start
5667 Start5670 Start
5666 Start5663 Start
5668 Start

5665 Start


5669 Start





5662 stop: found every range
5660 stop: found every range
5665 stop: found every range
5666 stop: found every range
5664 stop: found every range
5670 stop: found every range
5653 stop: found every range
5663 stop: found every range
5654 stop: found every range
5669 stop: found every range
5649 stop: found every range
5661 stop: found every range
5667 stop: found every range
5659 stop: found every range
5650 stop: found every range
5668 stop: found every range
5648 stop: found every range
5651 stop: found every range
5652 stop: found every range
5657 stop: found every range
5655 stop: found every range
5656 stop: found every range
5658 stop: found every range


## Merge non-AMPs and Check 

In [52]:
import glob

path = r'/mnt/vdb/thesis/non_amp/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

nonAMP_df = pd.concat(li, axis=0, ignore_index=True)

In [63]:
nonAMP_df.drop_duplicates(subset=['Sequence'],inplace=True)

,ID,Sequence,length,class
0,UniRef50_A0A1A8TYG8,RVIAENQFGQSPPSVPTEPLTTKEDKSVIRNYDEEVDEAREITKVE...,86,1
1,UniRef50_A0A6M0KXJ5,EKPTWLQRRIGSRLDDPEADRTASAGDWFSYQPGPSAAATGAAPVS...,110,1
2,UniRef50_E2AEH6,PSEIQIPKITYIEKNPVQTGILSRNAEEALQVLKKKYKKPKMSEKE...,373,1
3,UniRef50_A0A0C9RFC3,TCGYNALCAVVGHNPICSCPVRFTGDPFVSCTPIPMQESPPESRDP...,111,1
4,UniRef50_F6YX83,GQCLSVGNTYTCSCPTGYSGNNCQIPPNPCLSLPCLNGGQCQTVGA...,229,1
...,...,...,...,...
44341,UniRef50_T0J8N2,MLLASTDPAALLDMIGSYQVPSEMKWISSDER,32,1
44342,UniRef50_H0HGK9,MMGNARDPVTRSIIKLLIANFERYEAFVQKLV,32,1
44343,UniRef50_A0A2X3HFW3,MAGIWNWLNSTKLFDDGQINLSVLKQLIDFLF,32,1
44344,UniRef50_A0A2V3SLD6,MEPAFQVRKKGSQQKKDFKFKKLKKTDLKCEV,32,1


In [55]:
nonAMP_df.length.describe()

count    44346.000000
mean       112.571235
std        144.993806
min         11.000000
25%         29.000000
50%         70.000000
75%        124.000000
max       2337.000000
Name: length, dtype: float64

In [54]:
AMPs_df.length.describe()

count    44346.000000
mean       112.571235
std        144.993806
min         11.000000
25%         29.000000
50%         70.000000
75%        124.000000
max       2337.000000
Name: length, dtype: float64

In [ ]:
### non AMP to FASTA

with open("/mnt/vdb/thesis/non_amp/NonAMPs.final.ready.fasta", 'w') as file:
    for index, row in df.iterrows():
        header = row['ID']
        seq = row['Sequence']
        # print(header)
        file.write('>'+ header + '\n')
        file.write(seq + '\n')
           
print("--- End of Writing ----")

## Labeling 

In [57]:
# setup
nonAMP_df['class']=1
AMPs_df['class']=0

In [59]:
ready_df = pd.concat([AMPs_df,nonAMP_df], axis=0)
ready_df

,ID,Sequence,length,class
0,EN92515250|C|B3FJD7|phage,MAKKSVPLRKPAGSDGQGNIKVPGGPVVLDLGDFDDIFGPMESESP...,2337,0
1,EN54061055|C|F8SJ56|phage,MASKKTTLPKPKGINPQGSIVQLDLDDFDDLFDEDFGLPKKNSPYT...,2319,0
2,EN4815120|C|Q8SCY1|phage,MAKKVTLPKGQTGATGTTLGQAGNILDLSDVDDIFGDTPKAKKGSP...,2237,0
3,EN3016141|D|D2J8A7|bacteriocin,MAETIKGLRIDLSLKDMGVGRSITELKRSFRTLNSDLKVSSKNFEY...,1619,0
4,EN9175723|BD|Q93IM3|bacteriocin,MAKKKNTYKVPSIIALTLAGTALTTHHAQAADKTQDQSTNKNILND...,1564,0
...,...,...,...,...
44341,UniRef50_T0J8N2,MLLASTDPAALLDMIGSYQVPSEMKWISSDER,32,1
44342,UniRef50_H0HGK9,MMGNARDPVTRSIIKLLIANFERYEAFVQKLV,32,1
44343,UniRef50_A0A2X3HFW3,MAGIWNWLNSTKLFDDGQINLSVLKQLIDFLF,32,1
44344,UniRef50_A0A2V3SLD6,MEPAFQVRKKGSQQKKDFKFKKLKKTDLKCEV,32,1


In [60]:
ready_df.to_csv("/mnt/vdb/thesis/AMP_NonAMPs.ready.csv",sep=",", quotechar='"',index=False, quoting=csv.QUOTE_ALL)

In [61]:
ready_df.to_csv("../datasets/thesis/AMP_NonAMPs.ready.csv",sep=",", quotechar='"',index=False, quoting=csv.QUOTE_ALL)